In [0]:
path_bronze = 'abfss://bronze@sabneobankproject.dfs.core.windows.net/NeoBank_Modelling_Source.csv'
path_gold = 'abfss://gold@sabneobankproject.dfs.core.windows.net/NeoBank_Modelling.csv'

dfClientSRC = spark.read.csv(path_bronze, header=True, inferSchema=True)
dfClientTAG = spark.read.csv(path_gold, header=True, inferSchema=True)

In [0]:
dfClientSRC = dfClientSRC.filter(dfClientSRC.IsActiveMember==1)
#display(dfClientSRC)

In [0]:
dfClientSRC = dfClientSRC.withColumnRenamed("RowNumber", "RowNumber_SRC")\
    .withColumnRenamed("CustomerId", "CustomerId_SRC")\
    .withColumnRenamed("Surname", "Surname_SRC")\
    .withColumnRenamed("CreditScore", "CreditScore_SRC")\
    .withColumnRenamed("Geography", "Geography_SRC")\
    .withColumnRenamed("Gender", "Gender_SRC")\
    .withColumnRenamed("Age", "Age_SRC")\
    .withColumnRenamed("Tenure", "Tenure_SRC")\
    .withColumnRenamed("Balance", "Balance_SRC")\
    .withColumnRenamed("NumOfProducts", "NumOfProducts_SRC")\
    .withColumnRenamed("HasCrCard", "HasCrCard_SRC")\
    .withColumnRenamed("IsActiveMember", "IsActiveMember_SRC")\
    .withColumnRenamed("EstimatedSalary", "EstimatedSalary_SRC")\
    .withColumnRenamed("Exited", "Exited_SRC")
dfClientTAG = dfClientTAG.withColumnRenamed("RowNumber", "RowNumber_TAG")\
    .withColumnRenamed("CustomerId", "CustomerId_TAG")\
    .withColumnRenamed("Surname", "Surname_TAG")\
    .withColumnRenamed("CreditScore", "CreditScore_TAG")\
    .withColumnRenamed("Geography", "Geography_TAG")\
    .withColumnRenamed("Gender", "Gender_TAG")\
    .withColumnRenamed("Age", "Age_TAG")\
    .withColumnRenamed("Tenure", "Tenure_TAG")\
    .withColumnRenamed("Balance", "Balance_TAG")\
    .withColumnRenamed("NumOfProducts", "NumOfProducts_TAG")\
    .withColumnRenamed("HasCrCard", "HasCrCard_TAG")\
    .withColumnRenamed("IsActiveMember", "IsActiveMember_TAG")\
    .withColumnRenamed("EstimatedSalary", "EstimatedSalary_TAG")\
    .withColumnRenamed("Exited", "Exited_TAG")
#dfClientSRC.show()
#dfClientTAG.show()

In [0]:
dfClientJoin = dfClientSRC.join(dfClientTAG, dfClientSRC.CustomerId_SRC == dfClientTAG.CustomerId_TAG, how='left')
#display(dfClientJoin)

In [0]:
dfNewRegister = dfClientJoin.where(dfClientJoin.RowNumber_TAG.isNull())
dfNewRegister = dfNewRegister.select('RowNumber_SRC', 'CustomerId_SRC', 'Surname_SRC', 'CreditScore_SRC', 'Geography_SRC', 'Gender_SRC', 'Age_SRC', 'Tenure_SRC', 'Balance_SRC', 'NumOfProducts_SRC', 'HasCrCard_SRC', 'IsActiveMember_SRC', 'EstimatedSalary_SRC', 'Exited_SRC')



In [0]:
dfUpdate = dfClientJoin.where("""
    (CustomerId_SRC == CustomerId_TAG) and 
    (RowNumber_TAG != RowNumber_TAG or
     Surname_SRC != Surname_TAG or 
     CreditScore_SRC != CreditScore_TAG or 
     Geography_SRC != Geography_TAG or 
     Gender_SRC != Gender_TAG or 
     Age_SRC != Age_TAG or 
     Tenure_SRC != Tenure_TAG or 
     Balance_SRC != Balance_TAG or 
     NumOfProducts_SRC != NumOfProducts_TAG or 
     HasCrCard_SRC != HasCrCard_TAG or 
     IsActiveMember_SRC != IsActiveMember_TAG or 
     EstimatedSalary_SRC != EstimatedSalary_TAG or 
     Exited_SRC != Exited_TAG)
""")

dfUpdateInativos = dfUpdate

dfUpdate = dfUpdate.select('RowNumber_SRC', 'CustomerId_SRC', 'Surname_SRC', 'CreditScore_SRC', 'Geography_SRC', 'Gender_SRC', 'Age_SRC', 'Tenure_SRC', 'Balance_SRC', 'NumOfProducts_SRC', 'HasCrCard_SRC', 'IsActiveMember_SRC', 'EstimatedSalary_SRC', 'Exited_SRC')

dfUpdateInativos = dfUpdateInativos.select('RowNumber_TAG', 'CustomerId_TAG', 'Surname_TAG', 'CreditScore_TAG', 'Geography_TAG', 'Gender_TAG', 'Age_TAG', 'Tenure_TAG', 'Balance_TAG', 'NumOfProducts_TAG', 'HasCrCard_TAG', 'IsActiveMember_TAG', 'EstimatedSalary_TAG', 'Exited_TAG')



In [0]:
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import when
from pyspark.sql.functions import *

In [0]:
dfUpdateInativos = dfUpdateInativos.select('*').withColumn('IsActiveMember_TAG', when(dfUpdateInativos.IsActiveMember_TAG == '1', '0').otherwise(dfUpdateInativos.IsActiveMember_TAG))


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [0]:
dfmerge = dfClientTAG.unionAll(dfUpdateInativos)


In [0]:
dfmerge = dfmerge.withColumn('_rown_number', row_number().over(Window.partitionBy('CustomerId_TAG').orderBy('IsActiveMember_TAG')))


In [0]:
dfmerge = dfmerge.select('*').where('_rown_number == 1')

In [0]:
dfmerge = dfmerge.drop('_rown_number')

In [0]:
dfversionfinal = dfmerge.unionAll(dfNewRegister).unionAll(dfUpdate)

In [0]:
dfversionfinal.select('*').where('CustomerId_TAG == 15628319 or CustomerId_TAG == 215628319')

DataFrame[RowNumber_TAG: int, CustomerId_TAG: int, Surname_TAG: string, CreditScore_TAG: int, Geography_TAG: string, Gender_TAG: string, Age_TAG: int, Tenure_TAG: int, Balance_TAG: double, NumOfProducts_TAG: int, HasCrCard_TAG: int, IsActiveMember_TAG: bigint, EstimatedSalary_TAG: double, Exited_TAG: int]

In [0]:
dfversionfinal = dfversionfinal.withColumnRenamed("RowNumber_TAG", "RowNumber")\
    .withColumnRenamed("CustomerId_TAG", "CustomerId")\
    .withColumnRenamed("Surname_TAG", "Surname")\
    .withColumnRenamed("CreditScore_TAG", "CreditScore")\
    .withColumnRenamed("Geography_TAG", "Geography")\
    .withColumnRenamed("Gender_TAG", "Gender")\
    .withColumnRenamed("Age_TAG", "Age")\
    .withColumnRenamed("Tenure_TAG", "Tenure")\
    .withColumnRenamed("Balance_TAG", "Balance")\
    .withColumnRenamed("NumOfProducts_TAG", "NumOfProducts")\
    .withColumnRenamed("HasCrCard_TAG", "HasCrCard")\
    .withColumnRenamed("IsActiveMember_TAG", "IsActiveMember")\
    .withColumnRenamed("EstimatedSalary_TAG", "EstimatedSalary")\
    .withColumnRenamed("Exited_TAG", "Exited")

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|     1288|  15565701|     Ferri|        698|    Spain|Female| 39|     9|161993.89|            1|        0|             0|       90212.38|     0|
|     4199|  15565706|  Akobundu|        612|    Spain|  Male| 35|     1|      0.0|            1|        1|             1|       83256.26|     1|
|     7091|  15565714|  Cattaneo|        601|   France|  Male| 47|     1| 64430.06|            2|        0|             1|       96517.97|     0|
|     2021|  15565779|      Kent|        627|  Germany|Female| 30|     6| 57809.32|            1|        1|             0|  

In [0]:
dfversionfinal.coalesce(1).write.mode("overwrite").csv(path='abfss://silver@sabneobankproject.dfs.core.windows.net/NeoBank_Modelling', header=True)